In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import json
import time
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import sys
from pathlib import Path
module_path = Path('.').resolve().parent
if str(module_path) not in sys.path:
    sys.path.append(str(module_path))

In [2]:
TOP_S = 100
BATCH_SIZE = 32
EPOCHS = range(1)

In [3]:
""" Get data generator """
from src.datagens import DataSource, WSIDatagen
from src.config import SLIDE_DIR

DATASET_PATH = Path('/mnt/data/crc_ml/data/processed/Prostata/level1/r512px/c256px/t512px/no_overlap/datasets/1602530237.h5')
CONFIG_PATH = Path('/mnt/data/crc_ml/configs/level1.json')

with CONFIG_PATH.open('r') as config_json:
    config = json.load(config_json)
config['batch_size'] = BATCH_SIZE

train_ds = DataSource()
train_ds.add_dataset(DATASET_PATH, 'train')
test_ds = DataSource()
test_ds.add_dataset(DATASET_PATH, 'test')

dg = WSIDatagen(config, slide_dir=SLIDE_DIR)
train_gen = dg.get_sequential_generator(train_ds)
test_gen = dg.get_sequential_generator(test_ds)

In [4]:
""" Create model """
from src.training.models import PretrainedModel
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.losses import BinaryCrossentropy

model = PretrainedModel(config, conv_net=VGG16).model
loss_fn = BinaryCrossentropy(from_logits=False)
optimizer = Adam(1e-4)

[2020-10-28 02:39:01][INFO ][models] Applying L2 regularizer.
[2020-10-28 02:39:03][INFO ][models] Building vgg16 model.
[2020-10-28 02:39:03][INFO ][models] Model input size: [512, 512, 3]


In [ ]:
def forward_pass(gen):
    # Initialize aggregator
    results = tf.zeros(shape=(0,1))
    for batch in tqdm(gen):
        results = tf.concat([results, model(batch)], axis=0)
        
    _, idx = tf.math.top_k(tf.reshape(results,[-1]), k=TOP_S, sorted=False)
    return idx

@tf.function
def train_step(x, y, w):
    with tf.GradientTape() as tape:
        y_pred = model(x, training=True)
        loss_value = loss_fn(y, y_pred, sample_weight=w)
    grads = tape.gradient(loss_value, model.trainable_weights)
    return grads, loss_value
        
    
for epoch in EPOCHS:
    print(f'Epoch: {epoch}', flush=True)
    
    # Process all slides
    for slide_idx in range(len(test_gen.sampler.datasource)):
        grads = None
        start_time = time.time()

        # Prepare slide information
        slide_name = test_gen.prepare_slide(slide_idx)
        slide_label = int(slide_name[-1])
        
        # Get top predictions
        test_gen.batch_size = BATCH_SIZE
        top_indices = forward_pass(test_gen)
        
        test_gen.batch_size = 1
        for batch_id in tqdm(top_indices):
            # Prepare inputs
            x, _ = test_gen[batch_id]
            if slide_label == 0:
                y  = tf.zeros((x.shape[0], 1))
                w = tf.ones((x.shape[0], 1)) * 0.61
            else:
                y  = tf.ones((x.shape[0], 1))
                w = tf.ones((x.shape[0], 1)) * 2.87
            
            # Accumulate gradients
            if grads is None:
                grads, loss_val = train_step(x, y, w)
            else:
                new_grads, loss_val = train_step(x, y, w)
                grads = [g1+g2 for g1, g2 in zip(grads, new_grads)]
                
        # Performing update
        grads = [grad / (TOP_S*len(test_gen.sampler.datasource)) for grad in grads]
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
            
        end_time = time.time()
        print(f'{slide_name} ({end_time - start_time:.4f} sec) Loss: {loss_val:.2f}')

Epoch: 0


100%|██████████| 100/100 [00:08<00:00, 11.41it/s]


TP-2019_6785-06-0 (162.5075 sec) Loss: 0.17


100%|██████████| 100/100 [00:09<00:00, 10.92it/s]


TP-2019_6786-06-0 (229.7466 sec) Loss: 0.08


...

100%|██████████| 100/100 [00:11<00:00,  8.72it/s]


TP-2019_7362-03-0 (200.2371 sec) Loss: 1.37


  0%|          | 0/2363 [00:00<?, ?it/s]

TP-2019_7333-09-0 (151.8564 sec) Loss: 0.69


  4%|▍         | 95/2363 [00:09<03:09, 11.98it/s]